# Named Tensor

##### Suppose we have 3D tensor of preprocessed image and convert it to grey scale. 
##### The weights can be found using https://en.wikipedia.org/wiki/Luma_(video)

In [7]:
import torch
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [8]:
# here we pretend to have a batch of 2:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]
batch_t

tensor([[[[ 1.4100, -1.3315,  0.0405,  1.3549,  1.0871],
          [ 0.6455,  1.3359, -1.2811, -0.9266,  0.4168],
          [ 1.0768, -0.1804,  0.1696,  0.5665, -0.6313],
          [ 1.4229,  1.8907, -3.4759, -0.0354,  0.2929],
          [ 1.9119,  0.5833, -0.0109, -1.0450,  0.2057]],

         [[ 0.0319, -1.2357, -0.7350, -0.4682,  1.1221],
          [ 0.8661,  0.8631,  0.5096,  0.4544,  1.3784],
          [ 0.5872, -0.4808, -0.4743, -2.0545,  0.3749],
          [-0.1347, -0.3108, -0.9702, -1.2456,  1.4473],
          [-0.0213,  1.3130,  3.2319, -0.8864, -1.3342]],

         [[-0.7277, -0.5777,  0.2528,  0.5586,  0.4245],
          [-0.3194, -0.5288,  0.8137,  1.0721,  1.1479],
          [ 0.3406, -1.8036, -1.0298, -0.8594,  0.1502],
          [ 0.4280, -0.5345, -1.6014, -0.7111, -1.8620],
          [ 0.8920,  1.8290, -0.9395, -0.9176, -1.8625]]],


        [[[ 0.0230, -0.3859, -0.2691, -0.2171,  2.8639],
          [-0.1624,  0.3184,  0.8411,  1.6167,  0.8433],
          [-1.3253, -0.

##### So sometimes the RGB channels are in dimension 0, and sometimes they are in dimension 1. But we can generalize by counting from the end: they are always in dimension –3, the third from the end.

#### The lazy, unweighted mean can thus be written as follows

In [11]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

##### But now we have the weight, too. PyTorch will allow us to multiply things that are the same shape, as well as shapes where one operand is of size 1 in a given dimension. It also appends leading dimensions of size 1 automatically. This is a feature called broadcasting. batch_t of shape (2, 3, 5, 5) is multiplied by unsqueezed_weights of shape (3,1, 1), resulting in a tensor of shape (2, 3, 5, 5), from which we can then sum the third dimension from the end (the three channels):

In [12]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

##### As often in Python, broadcasting—a form of summarizing unnamed things—is done using three dots '...' ; but don’t worry too much about einsum , because we will not use it in the following

In [13]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

##### There's lot of bookkeeping involved so Named Tensors came to resue in pytorch version 1.3

### Tensor factory functions such as tensor and rand take a names argument. 
The names should be a sequence of strings:

In [14]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:848.)
  """Entry point for launching an IPython kernel.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [16]:
# When we already have a tensor and want to add names (but not change existing ones), we can call the method refine_names on it.
# Similar to indexing, the ellipsis ( ...) allows you to leave out any number of dimensions
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.name)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) None


In [17]:
# The method align_as returns a tensor with missing 
# dimensions added and existing ones permuted to the right order:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [20]:
# Functions accepting dimension arguments, like sum , also take named dimensions:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

#### Named tensors have the potential to eliminate many sources of alignment errors, which—if the PyTorch forum is any indication—can be a source of headaches. It will be interesting to see how widely they will be adopted.